Importation des modules

In [ ]:
import affichage_plotly_trajets as apt
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import affichage_total as at


Choix des paramètres

In [ ]:
nom_ville='Toulouse'  #Ville d'origine
minutes_max = 10000 #Période d'étude en minutes

Calcul des OD par mode (peut prendre du temps à tourner, env 8min)

Le code ci dessous renvoie un dataframe avec les O/D en Train/Avion et Voiture, depuis la ville "nom_ville". Pour chaque O/D, le tableau contient les info l'origine ("cityFrom", "lat_ori", "lon_ori"), de la station d'origine (gare pour les train, aeroport pour les avion, avec le nom de la station), les infos de la ville d'arrivée ("cityTo", "lat", "lon"), de la station d'arrivée. Aussi, on calcule les Temps de transit aller (temps entre centre-ville d'origine et station d'origine) et retour (Telos entre station d'arrivée et centre-ville de la ville d'arrivée), la distance du trajet ("Distance", en km). On calcule aussi des coordonnés "centrées" de chaque ville d'arrivée (calculées en faisant, =cordonnées d'arrivée-coordonnées de départ, qui sont les variables "lon_dest_centre" et "lat_dest_centre"), et les coordonnées proportionnels au temps de trajets (pour tracer le graphique des cercles concentriques).

In [ ]:
dest_reduit_total  = apt.affichage_plotlty(nom_ville=nom_ville, minutes_max=minutes_max)


dest_reduit_total.drop_duplicates(subset=['mode','cityTo','cityFrom'], keep='last', inplace=True)
dest_reduit_total.to_csv('Full_res/Resultats' + '_' + str(nom_ville) + '.csv')
dest_reduit_total


Si résultats déjà générés

In [ ]:
dest_reduit_total=pd.read_csv('Full_res/Resultats_'+str(nom_ville)+'.csv')


Réduction aux villes principales (+200k habitants)

On sélectionne les lignes du dataframe uniquement pour des destinations >200k habitants

In [ ]:
villes= ['Lille','Strasbourg','Paris','Brest','Bordeaux','Nantes', 'Nice','Lyon','Bayonne', 'Toulouse','Marseille','Rennes']
dest_reduit_total_red = dest_reduit_total.loc[dest_reduit_total['cityTo'].isin(villes)]
dest_reduit_total_red

Carte des destinations atteignables depuis la ville choisie (en bluen les trajets en train, en rouge les trajets en avion)

In [ ]:
at.affichage_total_df(dest_reduit_total)

Carte villes principales françaises atteignables depuis la ville choisie (en bluen les trajets en train, en rouge les trajets en avion)

In [ ]:
at.affichage_total_df(dest_reduit_total_red)

In [ ]:
apt.affichage_classique(dest_reduit_total_red)

Affichage des temps de trajets pour les villes principales en comparant les modes

In [1]:
liste_tps_train=[]
liste_tps_avion=[]
liste_tps_voit=[]
liste_tps_transit_train=[]
liste_tps_transit_avion=[]
liste_tps_transit_voit=[]
ville=villes
for ville_dest in villes:
    #test avion
    info_avion= dest_reduit_total_red[(dest_reduit_total_red['cityTo']==ville_dest) & (dest_reduit_total_red['mode']=='Avion')]
    if not info_avion.empty:
        tps_trajet_avion = info_avion['Tps_trajet_sec'].iloc[0]
        tps_transit_avion = info_avion['Tps_total_transit'].iloc[0]
        if tps_trajet_avion >0:
            liste_tps_avion.append(tps_trajet_avion)
        else:
            liste_tps_avion.append(0)
        if tps_transit_avion >0:
            liste_tps_transit_avion.append(tps_transit_avion)
        else: 
            liste_tps_transit_avion.append(0)
    else:
        liste_tps_avion.append(0)
        liste_tps_transit_avion.append(0)
    #test train
    info_train= dest_reduit_total_red[(dest_reduit_total_red['cityTo']==ville_dest) & (dest_reduit_total_red['mode']=='Train')]
    if not info_train.empty:
        tps_trajet_train = info_train['Tps_trajet_sec'].iloc[0]
        tps_transit_train = info_train['Tps_total_transit'].iloc[0]
        if tps_trajet_train >0:
            liste_tps_train.append(tps_trajet_train)
        else:
            liste_tps_train.append(0)
        if tps_transit_train >0:
            liste_tps_transit_train.append(tps_transit_train)
        else: 
            liste_tps_transit_train.append(0)
    else:
        liste_tps_train.append(0)
        liste_tps_transit_train.append(0)
    #test voiture
    info_voit= dest_reduit_total_red[(dest_reduit_total_red['cityTo']==ville_dest) & (dest_reduit_total_red['mode']=='Voiture')]
    if not info_voit.empty:
        tps_trajet_voit = info_voit['Tps_trajet_sec'].iloc[0]
        tps_transit_voit = info_voit['Tps_total_transit'].iloc[0]
        if tps_trajet_voit >0:
            liste_tps_voit.append(tps_trajet_voit)
        else:
            liste_tps_voit.append(0)
        if tps_transit_voit >0:
            liste_tps_transit_voit.append(tps_transit_voit)
        else: 
            liste_tps_transit_voit.append(0)
    else:
            liste_tps_voit.append(0)
            liste_tps_transit_voit.append(0)

x = np.arange(len(ville))
tps_voiture = [34, 56, 12]
tps_train = [12, 56, 78]
tps_avion = [14, 23, 45]
width = 0.3
  

plt.rcParams["figure.figsize"] = (10,10)
plt.grid()
plt.barh(x-0.3, liste_tps_avion, width, color='indianred',left=liste_tps_transit_avion)
plt.barh(x-0.3, liste_tps_transit_avion, width, color='firebrick')
plt.barh(x, liste_tps_train, width, color='lightgreen',left=liste_tps_transit_train)
plt.barh(x, liste_tps_transit_train, width, color='forestgreen')
plt.barh(x+0.3, liste_tps_voit, width, color='skyblue',left=liste_tps_transit_voit)
plt.barh(x+0.3, liste_tps_transit_voit, width, color='royalblue')
plt.yticks(x, villes)
plt.ylabel("Destinations depuis " + str(nom_ville))
plt.xlabel("Temps en secondes")
plt.legend(["Avion", "Transit aéroport" ,"Train",'Transit gare', "Voiture"])

plt.show()

NameError: name 'villes' is not defined